# ResourceStack

`ResourceStack` represents a collection of resources stacked together into a single
resource. It can grow along the x, y or z axis. This is useful when you want to
treat multiple resources as a single unit, for instance stacking lids vertically or
arranging plates side by side before placing them on the deck.

Because the stack is itself a `Resource`, it can be assigned to other resources or
the deck like any other labware. When the stack grows along the z-axis it behaves
like a traditional *stack* where items are added and removed from the top.

Below we demonstrate creating stacks in different orientations and interacting with
them.

In [29]:
from pylabrobot.resources import Resource, Plate, Lid, Coordinate
from pylabrobot.resources import ResourceStack

## Creating an empty stack
Pass the name and direction of growth (`"x"`, `"y"`, or `"z"`).

In [30]:
stack_x = ResourceStack("stack_x", "x")
stack_y = ResourceStack("stack_y", "y")
stack_z = ResourceStack("stack_z", "z")
(stack_x.children, stack_y.children, stack_z.children)

([], [], [])

## Stacking resources at construction time
You can also supply a list of resources which will be assigned immediately.

In [31]:
stack = ResourceStack(
    "stack",
    "x",
    [
        Resource("A", size_x=10, size_y=10, size_z=10),
        Resource("B", size_x=10, size_y=10, size_z=10),
    ],
)
([child.name for child in stack.children], stack.get_size_x())

(['A', 'B'], 20)

The total size along the x-axis equals the sum of the children sizes.

In [32]:
stack_y2 = ResourceStack(
    "stack_y2",
    "y",
    [
        Resource("A", size_x=10, size_y=10, size_z=10),
        Resource("B", size_x=10, size_y=10, size_z=10),
    ],
)
stack_y2.get_size_y()

20

## Adding and removing items
New items are positioned automatically at the edge returned by
`get_resource_stack_edge()`. When stacking in the z direction you can only remove
the current top item.

In [33]:
lid1 = Lid(name="L1", size_x=10, size_y=10, size_z=5, nesting_z_height=1)
lid2 = Lid(name="L2", size_x=10, size_y=10, size_z=5, nesting_z_height=1)
stack_z.assign_child_resource(lid1)
stack_z.assign_child_resource(lid2)
stack_z.get_top_item().name

'L2'

In [34]:
stack_z.unassign_child_resource(lid2)
stack_z.get_top_item().name

'L1'

Attempting to remove `lid1` now would raise a `ValueError` because it is not the
top item in this z-growing stack.

## Using a ResourceStack as a stacking area
A common use case is stacking plates next to a reader or washer. After placing a
plate on the stack you can retrieve it again using `get_top_item()`.

In [35]:
plate = Plate("p1", size_x=1, size_y=1, size_z=1, ordered_items={})
stacking_area = ResourceStack("stacking_area", "z")
stacking_area.assign_child_resource(plate)
stacking_area.get_top_item() is plate

True

When using a :class:`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` the
stack behaves just like any other resource:

```python
lh.move_plate(stacking_area.get_top_item(), plate_carrier[0])
```

This allows temporary storage of plates or lids during automated workflows.

## Moving plates and lids to the stacking area
The functions `move_lid()` and `move_plate()` can be used to move plates and lids to a ResourceStack during robot runtime.

Below is an example on the STARBackend:

In [36]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import LiquidHandlerChatterboxBackend, STARBackend
from pylabrobot.resources.hamilton import STARLetDeck

from pylabrobot.resources import (
  TIP_CAR_480_A00, 
  STF,
  PLT_CAR_L5AC_A00,
  Cor_96_wellplate_360ul_Fb,
  Cor_96_wellplate_2mL_Vb,
  ResourceStack,
)

Setup liquid handler and deck

In [37]:
lh = LiquidHandler(backend=LiquidHandlerChatterboxBackend(), deck=STARLetDeck())
await lh.setup()

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Resource waste_block was assigned to the liquid handler.


In [38]:
tip_car = TIP_CAR_480_A00(name="tip_carrier")
tip_rack = STF(name="tip_rack")
tip_car[0] = tip_rack
lh.deck.assign_child_resource(tip_car, rails=1)

Resource tip_carrier was assigned to the liquid handler.


In [39]:
plate_stack = ResourceStack("plate_stack", "z", [
    Cor_96_wellplate_2mL_Vb(name='stack_plate_1')
])

plt_car = PLT_CAR_L5AC_A00(name="plate_carrier")
plt_car[0] = plate_stack
plt_car[1] = plate_1 = Cor_96_wellplate_360ul_Fb(name="plate_1", with_lid=True)
plt_car[2] = plate_2 = Cor_96_wellplate_360ul_Fb(name="plate_2", with_lid=True)
lh.deck.assign_child_resource(plt_car, rails=8)

plate_1_lid = plate_1.lid
plate_2_lid = plate_2.lid

Resource plate_carrier was assigned to the liquid handler.


In [40]:
lh.summary()

Rail  Resource                      Type           Coordinates (mm)
(-6)  ├── trash_core96              Trash          (-58.200, 106.000, 229.000)
      │
(1)   ├── tip_carrier               TipCarrier     (100.000, 063.000, 100.000)
      │   ├── tip_rack              TipRack        (106.200, 073.000, 214.950)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(8)   ├── plate_carrier             PlateCarrier   (257.500, 063.000, 100.000)
      │   ├── plate_stack           ResourceStack  (261.500, 071.500, 184.950)
      │   │   ├── stack_plate_1     Plate          (261.500, 071.500, 184.950)
      │   ├── plate_1               Plate          (261.500, 167.500, 183.120)
      │   │   ├── plate_1_lid       Lid            (261.500, 167.500, 189.720)
      │   ├── plate_2               Plate          (261.500, 263.500, 183.120)
      │   │   ├── plate_2_lid       Lid            (261.500, 263.500, 189.720)
      │   ├── <empty>
      │   ├── <e

If the top of the stack is a plate without a lid, a lid moved with `move_lid()` to the stack will automatically become a child of the top plate.

Moving a plate with a lid with `move_plate()` will move both the plate and lid to the top of the stack.

In [ ]:
await lh.move_lid(plate_1_lid, plate_stack)
await lh.move_plate(plate_2, plate_stack)

Picking up resource: ResourcePickup(resource=Lid(name=plate_1_lid, location=Coordinate(000.000, 000.000, 006.600), size_x=127.76, size_y=85.48, size_z=8.9, category=lid), offset=Coordinate(x=0, y=0, z=0), pickup_distance_from_top=2.37, direction=<GripDirection.FRONT: 1>)
Dropping resource: ResourceDrop(resource=Lid(name=plate_1_lid, location=Coordinate(000.000, 000.000, 006.600), size_x=127.76, size_y=85.48, size_z=8.9, category=lid), destination=Coordinate(x=261.5, y=71.5, z=220.85), destination_absolute_rotation=Rotation(x=0, y=0, z=0), offset=Coordinate(x=0, y=0, z=0), pickup_distance_from_top=2.37, pickup_direction=<GripDirection.FRONT: 1>, drop_direction=<GripDirection.FRONT: 1>, rotation=0)
Resource plate_1_lid was unassigned from the liquid handler.
Resource plate_1_lid was assigned to the liquid handler.
Picking up resource: ResourcePickup(resource=Plate(name=plate_2, size_x=127.76, size_y=85.48, size_z=14.2, location=Coordinate(000.000, 000.000, -03.030)), offset=Coordinate(x=

Resource 'plate_2_lid' is very high on the deck: 245.25 mm. Be careful when traversing the deck.


Resource plate_2 was assigned to the liquid handler.


In [42]:
lh.summary()

Rail  Resource                       Type           Coordinates (mm)
(-6)  ├── trash_core96               Trash          (-58.200, 106.000, 229.000)
      │
(1)   ├── tip_carrier                TipCarrier     (100.000, 063.000, 100.000)
      │   ├── tip_rack               TipRack        (106.200, 073.000, 214.950)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(8)   ├── plate_carrier              PlateCarrier   (257.500, 063.000, 100.000)
      │   ├── plate_stack            ResourceStack  (261.500, 071.500, 184.950)
      │   │   ├── stack_plate_1      Plate          (261.500, 071.500, 184.950)
      │   │   │   ├── plate_1_lid    Lid            (261.500, 071.500, 220.850)
      │   │   ├── plate_2            Plate          (261.500, 071.500, 229.750)
      │   │   │   ├── plate_2_lid    Lid            (261.500, 071.500, 236.350)
      │   ├── plate_1                Plate          (261.500, 167.500, 183.120)
      │   ├── <empty>
     

**Warning:** Currently there are no checks in PyLabRobot for plate and lid compatibility when moving a lid to a ResourceStack. It is possible that the lid from one plate could be added to a plate of a different type on the ResourceStack with `move_lid()`. Users are responsible for making sure the lids and plates are compatible and will stack correctly.

TODO: Create a more permanent and robust fix: https://github.com/PyLabRobot/pylabrobot/pull/546#issuecomment-2945105532